#TRABAJO PRÁCTICO 1 - PANDAS
 - Alumno: Daniela Ojeda.
 - Padrón: 107690.
 - Cuatrimestre: 1C2023.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##EJERCICIO 4

In [ ]:
#Ejercicio4
'''La probabilidad de que la versión actual de un contenido fuera editada sin dejar comentario para usuarios que están logueados y que no están logueados'''
df_contents = pd.read_csv('/content/drive/MyDrive/orga/wikipedia/contents.csv')
n_sin_log = len(df_contents[df_contents['revisor_id'].isna()])
n_log = len(df_contents[df_contents['revisor_ip'].isna()])
n_sin_comentarios_no_log = len(df_contents[df_contents['revisor_id'].isna() & df_contents['revisor_comment'].isna()])
n_sin_comentarios_log = len(df_contents[df_contents['revisor_ip'].isna() & df_contents['revisor_comment'].isna()])
print(f'La probabilidad de que la versión actual de un contenido fuera editada sin dejar comentario para usuarios que están logueados es: {n_sin_comentarios_log/n_log}')
print(f'La probabilidad de que la versión actual de un conetenido fuera editada sin dejar comentario para usuarios que no están logueados es: {n_sin_comentarios_no_log/n_sin_log}')


La probabilidad de que la versión actual de un contenido fuera editada sin dejar comentario para usuarios que están logueados es: 0.09955855449191368
La probabilidad de que la versión actual de un conetenido fuera editada sin dejar comentario para usuarios que no están logueados es: 0.3987593578461045


##EJERCICIO 17

In [ ]:
#Ejercicio17
'''Utilice los textos del contenido para realizar consultas por texto utilizando las técnicas vistas en la clase de
NLP (BOW o TF-IDF) de modo que la query “retablo iglesia” devuelva alguna página acerca del retablo de alguna iglesia'''

!pip install nltk
from nltk import word_tokenize
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords


df_contents_text_sample = pd.read_csv('/content/drive/MyDrive/orga/wikipedia/contents_text_sample.csv')
query = "retablo iglesia"

stopwordsEsp = stopwords.words('spanish')
countVec = CountVectorizer(stop_words = stopwordsEsp)



from sklearn.neighbors import NearestNeighbors
buscaVecinos = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
text_no_na = df_contents_text_sample['text'].dropna()
text_no_na_lista = text_no_na.tolist()
matrizVectores = countVec.fit_transform(text_no_na_lista)
buscaVecinos.fit(matrizVectores)
busqueda = countVec.transform([query])
distancias, indices = buscaVecinos.kneighbors(busqueda, n_neighbors = 1)

print('')
print('Los resultados de la búsqueda para la query: {} utilizando BoW son los siguientes:'.format(query))
for i in indices[0]:
  print(text_no_na_lista[i])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.



Los resultados de la búsqueda para la query: retablo iglesia utilizando BoW son los siguientes:
#REDIRECCIÓN [[Retablo mayor de la iglesia de San Millán de los Balbases]]


##EJERCICIO 22

In [ ]:
#Ejercicio 22
"Elija su lugar favorito en el mundo y tome su latitud y longitud, ¿cuál es el título de la página de wikipedia más cercana?"
df_geotags = pd.read_csv('/content/drive/MyDrive/orga/wikipedia/geo_tags.csv')
df_contents = pd.read_csv('/content/drive/MyDrive/orga/wikipedia/contents.csv')
lugar_favorito = (51.4925, 7.451667)
df_geotags['distancia'] = ((df_geotags['gt_lat'] - lugar_favorito[0])**2 + (df_geotags['gt_lon'] - lugar_favorito[1])**2 )**0.5
mas_cercano = df_geotags.loc[df_geotags['distancia'].idxmin()]
fila_mas_cercano = df_contents.loc[df_contents['id'] == mas_cercano['gt_page_id']]
print('El título más cercano a mi lugar favorito es: {}'.format(fila_mas_cercano.iloc[0]['title']))

El título más cercano a mi lugar favorito es: Signal Iduna Park


##EJERCICIO 53

In [ ]:
#Ejercicio53
'''Si para un usuario tenemos la cantidad de acciones que realizó para cada tipo de log y la cantidad de veces que fue bloqueado: ¿Cuál es la acción que más y
menos correlaciona con ser bloqueado? ¿Qué acción correlaciona más con saber algo (babel>=0) de inglés?'''
#Aclaración: use solamente a los usuarios logueados
df_logs = pd.read_csv('/content/drive/MyDrive/orga/wikipedia/logs.csv')
df_languages = pd.read_csv('/content/drive/MyDrive/orga/wikipedia/languages.csv')
df_languages['babel_level'] = df_languages['babel_level'].replace('N', 4.5).astype(float)

<ipython-input-12-cab4242225c9>:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_logs = pd.read_csv('/content/drive/MyDrive/orga/wikipedia/logs.csv')


In [ ]:
#Obtengo la cantidad de acciones que realizo cada usuario para cada tipo de log
df_usuarios_acciones = df_logs.groupby(['contributor_username','logtype'])['action'].count().reset_index()
df_usuarios_acciones = df_usuarios_acciones.rename(columns={'contributor_username': 'usuario', 'action': 'cantidad_acciones'})
df_usuarios_acciones

,usuario,logtype,cantidad_acciones
0,! ! ! ! ! ! ! ! ! ! ! ! ! ! HAGGEⓇ? 123,newusers,1
1,! ! ! Histoire hebdomadaire de la patrie ! ! !,newusers,1
2,! Bikkit !,newusers,1
3,! DanSkammelsrod !,newusers,1
4,! Gabriel !,newusers,1
...,...,...,...
6387455,𝚕𝚒𝚝𝚢 𝚌𝚟18,newusers,1
6387456,😂,newusers,1
6387457,🤖,newusers,1
6387458,𠔻𪚥,newusers,1


In [ ]:
#Obtengo la cantidad de veces que fue bloqueado un usuario
df_usuarios_bloqueados = df_logs[df_logs['action'] == 'block']['title'].value_counts().reset_index()
df_usuarios_bloqueados = df_usuarios_bloqueados.rename(columns={'index': 'usuario', 'title': 'cantidad_bloqueos'})
df_usuarios_bloqueados['usuario'] = df_usuarios_bloqueados['usuario'].str[8:]
df_usuarios_bloqueados

,usuario,cantidad_bloqueos
0,80.58.205.33,33
1,80.58.205.37,32
2,80.58.205.44,32
3,Antoine,32
4,80.58.205.43,32
...,...,...
299761,79.151.186.51,1
299762,79.148.141.112,1
299763,98.199.59.122,1
299764,201.215.233.87,1


In [ ]:
#Obtengo la accción que más y menos correlaciona con ser bloqueado
df_logs = df_logs.rename(columns={'contributor_username': 'usuario'})
df_merge = pd.merge(df_logs, df_usuarios_bloqueados, on='usuario', how='inner')
correlaciones = df_merge.groupby('action')[['usuario', 'cantidad_bloqueos']].corr(numeric_only=True)
correlaciones_ordenadas = correlaciones['cantidad_bloqueos'].sort_values(ascending=False)
print(f'La acción que más correlaciona con ser bloqueado es: {correlaciones_ordenadas.index[0][0]}')
print(f'La acción que menos correlaciona con ser bloqueado es: {correlaciones_ordenadas.index[-1][0]}')

La acción que más correlaciona con ser bloqueado es: add
La acción que menos correlaciona con ser bloqueado es: usergroups


In [ ]:
#Obtengo la acción que correlaciona más con saber algo (babel>=0) de inglés
df_languages = df_languages.rename(columns={'babel_user': 'contributor_id'})
df_usuarios_ingles = df_languages[(df_languages['babel_lang'] == 'en') & (df_languages['babel_level'] >= 0)]
df_usuarios_ingles_acciones = pd.merge(df_usuarios_ingles, df_logs, on='contributor_id', how='inner')
correlaciones = df_usuarios_ingles_acciones.groupby('action')[['babel_level', 'contributor_id']].corr()
correlaciones_ordenadas = correlaciones['babel_level'].sort_values(ascending=False)
print(f'La acción que más correlaciona con saber algo (babel>=0) de inglés: {correlaciones_ordenadas.index[0][0]}')

La acción que más correlaciona con saber algo (babel>=0) de inglés: add
